# Analysis of Land Registry data on titles registered to individuals with an overseas correspondence address

This notebook analyses data on the number of Land Registry titles in England & Wales owned by individuals with an overseas correspondence address between 2010 and 2021.

This data is interesting beacuse it's a proxy (see below) for overseas citizens buying second homes or buying investment property in England & Wales, which otherwise we don't have any statistics on.

**Data source**

This data was obtained via Freedom of Information requests:
- data for [Jan 2010, 2012 and 2014](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_3)
- data for [Jan 2016 and 2018](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_2)
- data for [Jan 2020](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ)
- data for [Aug 2021](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_4).

**Technical notes on the data**

The data is titles owned by individuals "with an overseas correspondence address". The Land Registry [asks for a correspondence address](https://www.gov.uk/government/publications/registered-titles-whole-transfer-tr1/guidance-completing-form-tr1-for-the-transfer-of-registered-property#panel-6-address) when a property is registered, so it can write to that address if an attempt is made to change the entry on the register.

Normally people just use the address of the property - so an overseas correspondence address is a good indication that they don't mainly live at the property, either because it's a second home, or because they rent it. (It's possible that a UK resident citizen might register an overseas correspondence address, but that would still indicate they intend to spend a lot of time overseas.)

And some other important points:

- This data is for titles owned by individuals, not companies. Data on titles owned by overseas companies is published separately.
- Titles aren't necessarily residential property, they can also be commercial property or land.
- Titles can be leasehold or freehold. 
- One title can cover multiple properties, though this is rare.
- Titles are registered in the name of the legal owner, not beneficial owner: titles can be registered in the names of individuals who hold it in trust for others, for example. 
- The data is stock, not flow, i.e. it is the total number of properties owned at each point in time, not sales. 

In [2]:
import glob
import pandas as pd

## Headline findings

In [16]:
df_by_country = pd.read_csv("./data/by_country.csv")

Totals by year.

In [15]:
years = ["2010_01", "2012_01", "2014_01", "2016_01", "2018_01", "2020_01", "2021_08"]
for y in years:
    print(y, df_by_country['TITLES_%s' % y].sum())

2010_01 87810
2012_01 120042
2014_01 152303
2016_01 184000
2018_01 215569
2020_01 239654
2021_08 247016


Increases from year to year. Some important tax events during this period:

1. In 2013 the Government introduced an [annual tax on UK residential property owned by a company](https://www.gov.uk/guidance/annual-tax-on-enveloped-dwellings-the-basics) worth over £2 million, and over the following years it reduced the threshold for this tax. This will have increased the incentives for owning as an individual rather than via a company.
2. In April 2017, the Government [brought all UK residential property within the scope of inheritance tax](https://www.cbw.co.uk/2021/05/reforms-to-the-taxation-of-non-doms-how-have-the-changes-introduced-in-2017-affected-non-doms-living-in-the-uk-today/), again reducing the incentives to own via an overseas company - I think.
2. In April 2021, the Government introduced a 2% surcharge on stamp duty for property acquired by overseas tax residents, though it's probably too early to see any effect of this yet.

NB I have [checked](https://www.whatdotheyknow.com/alaveteli_pro/info_requests/titles_owned_by_overseas_individ_3) that these numbers do mean all past titles, including ones that have since been sold - obviously if only current titles were included, the numbers could only rise :)

There is a slight slowing between 2018 and 2020, perhaps the effects of tax changes in 2017, and a much slower rise during the pandemic, as you'd expect. 

In [25]:
for i, y in enumerate(years):
    if i+1 != len(years):
        print("Change from %s to %s:" % (years[i], years[i+1]),
            df_by_country['TITLES_%s' % years[i+1]].sum() - df_by_country['TITLES_%s' % years[i]].sum())

Change from 2010_01 to 2012_01: 32232
Change from 2012_01 to 2014_01: 32261
Change from 2014_01 to 2016_01: 31697
Change from 2016_01 to 2018_01: 31569
Change from 2018_01 to 2020_01: 24085
Change from 2020_01 to 2021_08: 7362


We should normalise the past number by the total number of titles.

There were [25,945,597 registered titles in 2021](https://www.gov.uk/government/publications/hm-land-registry-annual-report-and-accounts-2020-to-2021) and ["22.8 million" registered titles in 2010](https://www.gov.uk/government/publications/land-registry-annual-report-and-accounts-2010-to-2011). This tells us that the percentage of all titles registered to overseas individuals has gone from 0.39% to 0.95% over the period.

In [49]:
print("{:.2f}%".format(df_by_country.TITLES_2010_01.sum() / 22800000 * 100.0), "of titles in 2010")
print("{:.2f}%".format(df_by_country.TITLES_2021_08.sum() / 25945597 * 100.0), "of titles in 2021")

0.39% of titles in 2010
0.95% of titles in 2021


Overseas companies [owned](https://www.gov.uk/guidance/hm-land-registry-overseas-companies-that-own-property-in-england-and-wales#access-the-data) 94,712 titles in August 2021, so altogether, overseas companies and individuals owned 1.32% of titles in 2021, or 1 in 75 titles. 

In [50]:
print("{:.2f}%".format((df_by_country.TITLES_2021_08.sum() + 94712) / 25945597 * 100.0), "of titles in 2021")

1.32% of titles in 2021


There are 214 countries represented (the data includes some things that aren't really countries, like the Crown Dependencies and overseas territories).

In [51]:
len(df_by_country)

218

The top 10 countries account for more than 50% of total titles owned overseas, and the top 20 for more than 75%.

In [27]:
print(df_by_country.sort_values("TITLES_2021_08", ascending=False).head(10).TITLES_2021_08.sum())
print(df_by_country.sort_values("TITLES_2021_08", ascending=False).head(12).TITLES_2021_08.sum())
print(df_by_country.sort_values("TITLES_2021_08", ascending=False).head(20).TITLES_2021_08.sum())
print(df_by_country.TITLES_2021_08.sum())

135956
151809
193968
247016


## Countries with most owners

The top 20 countries make up 75% of the total, so let's have a look at them here. These fall into a few distinct groups:

- South East Asia: Hong Kong, Singapore, Malaysia, China, Taiwan - lowish numbers in 2010, bigger rises over the period than other group
- Crown dependencies and British overseas territories (i.e. tax havens): Jersey, Guernsey, Isle of Man, British Virgin Islands - healthy numbers already by 2010, generally bigger rises earlier in the decade, still rising but slowing now.
- Countries with historic links to the UK: the US, Australia, South Africa, France - quite stong rises. Just people buying second homes?
- The Middle East: UAE, Saudi Arabia, Qatar - also big rises
- Ireland (combined with Eire at data processing stage) - outlier in having a big drop over the period. People having to sell up because of the financial crisis? Or an ageing population? I'd like to understand this better. (NB there aren't big drops anywhere else, though there is a small fall in North Korea. "Channel Islands" rises and falls over the period, though that may just reflect addressing conventions?)

In [53]:
df_by_country.sort_values("TITLES_2021_08", ascending=False).head(20)

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,HONG KONG,2170,4300,7278,11731,17382,20656,23524,21354,984.055300
1,JERSEY,6125,9062,11983,15791,16977,18194,18493,12368,201.926531
2,SINGAPORE,7323,9928,12735,15016,16822,18201,18165,10842,148.054076
3,UNITED STATES OF AMERICA,5292,6622,7922,9753,11152,12651,13076,7784,147.089947
4,IRELAND,18759,18743,17075,14781,13242,12457,12056,-6703,-35.732182
5,GUERNSEY,3783,6584,8550,10045,10874,11405,11594,7811,206.476342
6,UNITED ARAB EMIRATES,2288,3139,4478,6080,8512,10415,10914,8626,377.010490
7,MALAYSIA,1987,3886,5302,6952,9621,10188,10177,8190,412.179165
8,ISLE OF MAN,5334,7358,8593,8786,9331,9574,9338,4004,75.065617
9,AUSTRALIA,2848,4171,5461,6353,7415,8245,8619,5771,202.633427


For each major group in the top 12, print the total now and the net increase since 2010.

In [26]:
groups = [
    ["HONG KONG", "SINGAPORE", "MALAYSIA"],
    ["JERSEY", "GUERNSEY", "ISLE OF MAN", "BRITISH VIRGIN ISLANDS"],
    ["UNITED STATES OF AMERICA", "IRELAND", "AUSTRALIA"],
    ["UNITED ARAB EMIRATES", "KUWAIT"],
]
for group in groups:
    print("%s:" % group)
    print(df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2021_08.sum())
    print(df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2021_08.sum()\
            -df_by_country[df_by_country.COUNTRY.isin(group)].TITLES_2010_01.sum())

['HONG KONG', 'SINGAPORE', 'MALAYSIA']:
51866
40386
['JERSEY', 'GUERNSEY', 'ISLE OF MAN', 'BRITISH VIRGIN ISLANDS']:
47323
29527
['UNITED STATES OF AMERICA', 'IRELAND', 'AUSTRALIA']:
33751
6852
['UNITED ARAB EMIRATES', 'KUWAIT']:
18869
15099


There are a small number in the Channel Islands separate from Jersey/Guernsey too, but not enough to worry about too much.

In [114]:
df_by_country[df_by_country.COUNTRY == "CHANNEL ISLANDS"]

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
34,CHANNEL ISLANDS,1175,1465,2060,1956,1447,1300,1180,5,0.425532


Next, which countries contributed most the increase between 2010 and 2021?

The absolute number of titles owned overseas rose by just under 160k over the period.

In [56]:
df_by_country.abs_change_10_21.sum()

159206

Just 12 countries were responsible for more than two-thirds of the increase.

In [57]:
df_by_country.sort_values("abs_change_10_21", ascending=False).head(12).abs_change_10_21.sum()

106998

There were particularly large rises in:
    
- Hong Kong, Singapore, Malaysia and China, with 47,213 new titles - the number of Hong Kong titles rose 10 times, China titles rose 25 times
- UAE, Kuwait, and Saudi Arabia, with 20,707 new titles - all up over 4 times

And also big rises (from a higher original base) in: 

- Jersey, Guernsey, and British Virgin Islands, with 25,523 new titles - all up roughly 3 times
- The US and Australia, with 13,555 new titles - up 2-3 times

In [115]:
df_by_country.sort_values("abs_change_10_21", ascending=False)[country_cols].head(12)

,COUNTRY,abs_change_10_21,TITLES_2021_08,TITLES_2010_01
0,HONG KONG,21354,23524,2170
1,JERSEY,12368,18493,6125
2,SINGAPORE,10842,18165,7323
6,UNITED ARAB EMIRATES,8626,10914,2288
7,MALAYSIA,8190,10177,1987
5,GUERNSEY,7811,11594,3783
3,UNITED STATES OF AMERICA,7784,13076,5292
13,CHINA,6827,7078,251
10,KUWAIT,6473,7955,1482
9,AUSTRALIA,5771,8619,2848


Check out the countries with the biggest increases, relative to their starting point.

In [59]:
df_by_country[df_by_country.TITLES_2021_08 > 500].sort_values("percent_change_10_21", ascending=False).head()

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
33,TAIWAN,29,59,100,431,1005,1155,1230,1201,4141.379310
13,CHINA,251,463,896,1793,3451,5932,7078,6827,2719.920319
35,RUSSIAN FEDERATION,86,179,404,710,985,1128,1127,1041,1210.465116
0,HONG KONG,2170,4300,7278,11731,17382,20656,23524,21354,984.055300
18,QATAR,302,483,1400,1852,2475,3049,3189,2887,955.960265


Finally, the number of titles fell over the period in Ireland and North Korea. This is also reassuring proof that we haven't just been given numbers for current titles. 

In [60]:
df_by_country[df_by_country.TITLES_2021_08 > 0].sort_values("abs_change_10_21").head(5)

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
4,IRELAND,18759,18743,17075,14781,13242,12457,12056,-6703,-35.732182
132,THE PEOPLES REPUBLIC OF KOREA,61,56,40,32,23,19,15,-46,-75.409836
88,ICELAND,99,94,86,89,87,89,90,-9,-9.090909
124,MARSHALL ISLANDS,27,37,31,39,30,24,21,-6,-22.222222
206,MONTSERRAT,4,4,3,3,2,1,1,-3,-75.000000


## By district

Ideally we might normalise this by the total titles per district, but this isn't something Land Registry publish, I think - there are stats on the [total dwelling stock](https://www.gov.uk/government/statistical-data-sets/live-tables-on-dwelling-stock-including-vacants) per district, but dwelling stock isn't the same as titles.

In [61]:
df_dist = pd.read_csv("by_dist.csv")
df_dist.head(50)

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11676,12104,8520,237.723214
1,LIVERPOOL,1964,2311,3272,4355,6029,7511,7925,5961,303.513238
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7016,7547,5669,301.863685
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7021,7073,4591,184.971797
4,BUCKINGHAMSHIRE,1847,3696,4842,6155,6299,6568,6609,4762,257.823498
5,SALFORD,1681,2125,2264,2702,3720,4880,5294,3613,214.931588
6,WANDSWORTH,1309,1994,2456,3131,4246,4903,5110,3801,290.374332
7,HOUNSLOW,4133,4251,4480,4612,4784,4885,4923,790,19.114445
8,MANCHESTER,2484,2807,2887,3287,4104,4341,4793,2309,92.954911
9,SOUTHWARK,747,1202,1626,2084,3064,3736,3796,3049,408.165997


In [62]:
len(df_dist)

331

London makes up about a third of the total, and about a third of the rise since 2010.

In [91]:
london_boroughs = ['BARKING AND DAGENHAM', 'BARNET',  'BEXLEY', 'BRENT', 
                   'BROMLEY', 'CAMDEN', 'CITY OF LONDON', 'CROYDON', 'EALING', 
                  'ENFIELD', 'GREENWICH', 'HACKNEY', 'HAMMERSMITH AND FULHAM',
                  'HARINGAY', 'HARROW', 'HARINGEY', 'HAVERING', 'HILLINGDON', 'HOUNSLOW', 
                  'ISLINGTON', 'KENSINGTON AND CHELSEA', 'KINGSTON UPON THAMES', 
                  'LAMBETH', 'LEWISHAM', 'MERTON', 'NEWHAM', 'REDBRIDGE', 
                  'RICHMOND UPON THAMES', 'SOUTHWARK', 'SUTTON', 'TOWER HAMLETS', 
                  'WALTHAM FOREST', 'WANDSWORTH', 'CITY OF WESTMINSTER']
df_london = df_dist[df_dist.DIST.isin(london_boroughs)]
df_london.sort_values("abs_change_10_21", ascending=False).head()

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21,delta_2020_2016
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11676,12104,8520,237.723214,8925.0
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7016,7547,5669,301.863685,3185.0
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7021,7073,4591,184.971797,2898.0
6,WANDSWORTH,1309,1994,2456,3131,4246,4903,5110,3801,290.374332,4335.0
9,SOUTHWARK,747,1202,1626,2084,3064,3736,3796,3049,408.165997,1892.0


In [84]:
print(df_london.TITLES_2021_08.sum())
print(df_london.TITLES_2010_01.sum())
print(df_london.abs_change_10_21.sum())

85451
26470
58981


Other major city centres account for about 31,000 titles. (For the purposes of this analysis, Birmingham is in the North :))

In [116]:
northern_cities = ['LIVERPOOL', 'SEFTON', 'MANCHESTER', 'SALFORD', 'TRAFFORD',
                'LEEDS', 'BIRMINGHAM', 'SHEFFIELD', 'CITY OF NOTTINGHAM', 'BRADFORD']
df_cities_northern = df_dist[df_dist.DIST.isin(northern_cities)]
print(df_cities_northern.TITLES_2021_08.sum())
print(df_cities_northern.abs_change_10_21.sum())

33398
22483


Which districts have seen the most dramatic rises?

The same we've already looked at, plus some Home Counties boroughs, in Buckinghamshire, Tandridge (in east Surrey), North Northamptonshire and Central Bedfordshire. Buckinghamshire is just a big area. 

Just outside the top 20 we also have Leeds, Sefton, Sheffield, Trafford and Nottingham. 

We also have Cornwall, Bournemouth, and Brighton in the top 40. 

In [67]:
df_dist.sort_values("abs_change_10_21", ascending=False).head(40)

,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,CITY OF WESTMINSTER,3584,5693,8713,10438,10856,11676,12104,8520,237.723214
1,LIVERPOOL,1964,2311,3272,4355,6029,7511,7925,5961,303.513238
2,TOWER HAMLETS,1878,3490,4452,4968,6142,7016,7547,5669,301.863685
4,BUCKINGHAMSHIRE,1847,3696,4842,6155,6299,6568,6609,4762,257.823498
3,KENSINGTON AND CHELSEA,2482,4060,5131,5960,6594,7021,7073,4591,184.971797
6,WANDSWORTH,1309,1994,2456,3131,4246,4903,5110,3801,290.374332
5,SALFORD,1681,2125,2264,2702,3720,4880,5294,3613,214.931588
9,SOUTHWARK,747,1202,1626,2084,3064,3736,3796,3049,408.165997
15,TANDRIDGE,201,603,2298,2411,2950,3012,3014,2813,1399.502488
12,HAMMERSMITH AND FULHAM,748,1205,1667,2133,2767,3302,3533,2785,372.326203


In [68]:
df_dist.sort_values("TITLES_2021_08", ascending=False).head(15).TITLES_2021_08.sum()

82599

In [70]:
df_dist.sort_values("abs_change_10_21", ascending=False).head(15).abs_change_10_21.sum()

58303

Now we want to dive into what's going on in those districts. 

In [71]:
top_districts = df_dist.sort_values("abs_change_10_21", ascending=False).DIST.head(10)
top_districts

0        CITY OF WESTMINSTER
1                  LIVERPOOL
2              TOWER HAMLETS
4            BUCKINGHAMSHIRE
3     KENSINGTON AND CHELSEA
6                 WANDSWORTH
5                    SALFORD
9                  SOUTHWARK
15                 TANDRIDGE
12    HAMMERSMITH AND FULHAM
Name: DIST, dtype: object

## By district and local authority

The files are sorted by the absolute change between 2010 and 2021. Where are the hotspots?

- Hong Kong and Jersey are big factors both in London and Northern cities
- This shows us that there's been a large number of Singapore-owned titles in Hounslow since at least 2010 - maybe Heathrow?
- It also shows sudden bursts of activity in particular districts e.g. Kuwaiti owners in Buckinghamshire?! - is that a particular development, or marketing push?

In [72]:
df_dc = pd.read_csv("df_by_country_and_dist.csv")
df_dc.head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,SINGAPORE,HOUNSLOW,3685,3665,3669,3684,3691,3691,3692,7,0.189959
1,KUWAIT,BUCKINGHAMSHIRE,555,578,1099,2108,2168,2219,2219,1664,299.819820
2,HONG KONG,TOWER HAMLETS,185,689,1020,1183,1544,1781,1959,1774,958.918919
3,JERSEY,CITY OF WESTMINSTER,518,827,1919,2306,1505,1558,1564,1046,201.930502
4,HONG KONG,SALFORD,33,55,98,392,879,1229,1562,1529,4633.333333
5,MALAYSIA,NORTH NORTHAMPTONSHIRE,0,1,3,95,1062,1428,1465,1465,inf
6,HONG KONG,MANCHESTER,45,58,76,327,954,967,1270,1225,2722.222222
7,HONG KONG,CITY OF WESTMINSTER,132,254,455,680,952,1133,1232,1100,833.333333
8,BRITISH VIRGIN ISLANDS,CITY OF WESTMINSTER,443,817,1023,1193,1208,1179,1190,747,168.623025
9,HONG KONG,LIVERPOOL,14,36,175,432,815,1029,1097,1083,7735.714286


This lets us dive into the patterns for particular countries, or districts, a bit more. 

For example, which countries have led the rise in London?

In [119]:
df_dc_london = df_dc[df_dc.DIST.isin(london_boroughs)]
country_cols = ['COUNTRY', 'abs_change_10_21', 'TITLES_2021_08', 'TITLES_2010_01']
df_dc_london.groupby("COUNTRY").sum().reset_index().\
    sort_values("abs_change_10_21", ascending=False)[country_cols].head(15)

,COUNTRY,abs_change_10_21,TITLES_2021_08,TITLES_2010_01
69,HONG KONG,10501,11642,1141
146,SINGAPORE,5884,10125,4241
35,CHINA,4307,4412,105
83,JERSEY,3762,5960,2198
25,BRITISH VIRGIN ISLANDS,3166,4684,1518
100,MALAYSIA,3152,3594,442
174,UNITED ARAB EMIRATES,2848,3475,627
175,UNITED STATES OF AMERICA,2436,3708,1272
67,GUERNSEY,2281,3573,1292
140,SAUDI ARABIA,1486,1726,240


In [120]:
df_dc_northern = df_dc[df_dc.DIST.isin(northern_cities)]
df_dc_northern.groupby("COUNTRY").sum().reset_index().\
    sort_values("abs_change_10_21", ascending=False)[country_cols].head(15)

,COUNTRY,abs_change_10_21,TITLES_2021_08,TITLES_2010_01
59,HONG KONG,5934,6094,160
152,UNITED ARAB EMIRATES,1834,2014,180
127,SINGAPORE,1701,1733,32
32,CHINA,1372,1386,14
73,JERSEY,1367,1688,321
23,BRITISH VIRGIN ISLANDS,1056,1183,127
90,MALAYSIA,994,1009,15
123,SAUDI ARABIA,909,927,18
77,KUWAIT,790,804,14
57,GUERNSEY,710,1094,384


And where are the districts that have seen the number of Hong Kong owners rise the most?

In [108]:
dist_cols = ['DIST', 'abs_change_10_21', 'TITLES_2021_08', 'TITLES_2010_01']
df_dc[df_dc.COUNTRY == "HONG KONG"].head(10)[dist_cols]

,DIST,abs_change_10_21,TITLES_2021_08,TITLES_2010_01
2,TOWER HAMLETS,1774,1959,185
4,SALFORD,1529,1562,33
6,MANCHESTER,1225,1270,45
7,CITY OF WESTMINSTER,1100,1232,132
9,LIVERPOOL,1083,1097,14
16,SOUTHWARK,746,793,47
20,NEWHAM,731,741,10
21,BIRMINGHAM,708,728,20
31,CAMDEN,540,570,30
32,KENSINGTON AND CHELSEA,455,564,109


In [74]:
df_dc[df_dc.DIST == "SALFORD"].head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
4,HONG KONG,SALFORD,33,55,98,392,879,1229,1562,1529,4633.333333
42,IRELAND,SALFORD,1075,1064,953,770,608,524,487,-588,-54.697674
50,BRITISH VIRGIN ISLANDS,SALFORD,3,4,25,128,83,456,452,449,14966.666667
97,CHINA,SALFORD,2,2,8,44,132,247,302,300,15000.000000
105,UNITED ARAB EMIRATES,SALFORD,26,30,41,82,226,272,289,263,1011.538462
116,SINGAPORE,SALFORD,5,11,100,147,179,227,273,268,5360.000000
153,ISLE OF MAN,SALFORD,382,641,549,416,295,246,224,-158,-41.361257
180,KUWAIT,SALFORD,3,2,4,32,126,178,181,178,5933.333333
209,SAUDI ARABIA,SALFORD,1,1,7,21,102,146,154,153,15300.000000
354,GUERNSEY,SALFORD,10,96,112,110,98,100,101,91,910.000000


In [90]:
df_dc[df_dc.DIST == "CITY OF WESTMINSTER"].sort_values("abs_change_10_21", ascending=False).head(10)

,COUNTRY,DIST,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
7,HONG KONG,CITY OF WESTMINSTER,132,254,455,680,952,1133,1232,1100,833.333333
3,JERSEY,CITY OF WESTMINSTER,518,827,1919,2306,1505,1558,1564,1046,201.930502
8,BRITISH VIRGIN ISLANDS,CITY OF WESTMINSTER,443,817,1023,1193,1208,1179,1190,747,168.623025
23,SINGAPORE,CITY OF WESTMINSTER,55,166,317,481,600,672,679,624,1134.545455
14,GUERNSEY,CITY OF WESTMINSTER,322,590,829,869,943,875,862,540,167.701863
46,UNITED ARAB EMIRATES,CITY OF WESTMINSTER,60,103,151,225,354,443,475,415,691.666667
98,MALAYSIA,CITY OF WESTMINSTER,27,97,161,216,281,299,302,275,1018.518519
56,UNITED STATES OF AMERICA,CITY OF WESTMINSTER,169,202,262,279,360,404,416,247,146.153846
132,CHINA,CITY OF WESTMINSTER,6,16,29,60,136,201,252,246,4100.000000
129,KUWAIT,CITY OF WESTMINSTER,12,19,47,96,135,221,255,243,2025.000000
